In [6]:
from CONSTANT import *
from tools import *
import pandas as pd
import numpy as np
from torch.utils.data import (TensorDataset, DataLoader, SequentialSampler, WeightedRandomSampler)

import torch

from torch import nn

use_cuda = False
valid='loso'

device = torch.device('cuda' if torch.cuda.is_available() and use_cuda else 'cpu')
print('using', device)

using cpu


In [8]:
class DataPrepare(object):
    def __init__(self, tag, data, train_index, test_index, batch_size=64,):

        X, y = join_signals(data, target=tag)
        xtrain, ytrain, xtest, ytest = X[train_index], y[train_index], X[test_index], y[test_index]
        print(xtrain.shape, ytrain.shape, xtest.shape, ytest.shape)
        
        self.xtrain = torch.from_numpy(xtrain).to(device)
        self.xtest = torch.from_numpy(xtest).to(device)
        
        self.ytrain = torch.from_numpy(ytrain).to(device)
        self.ytest = torch.from_numpy(ytest).to(device) 
        
        print(self.xtrain.isnan().any(), self.xtest.isnan().any(), 
              self.ytrain.isnan().any(), self.ytest.isnan().any(),)
        
        self.batch_size = batch_size
        
    
    def get_data(self):
        train_data = TensorDataset(self.xtrain, self.ytrain)
        test_data = TensorDataset(self.xtest, self.ytest)
        
        train_sampler = SequentialSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=self.batch_size, drop_last=True)
        
        test_sampler = SequentialSampler(test_data)
        test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=self.batch_size, drop_last=True)

        return train_dataloader, test_dataloader

In [12]:
spliter = load_model(r'./processed_signal/HKU956/400_4s_step_2s_spliter.pkl')
data = pd.read_pickle(r'./processed_signal/HKU956/400_4s_step_2s.pkl')
for k in spliter[valid]:
    train_index = k['train_index']
    test_index = k['test_index']
    break

In [13]:
dataprepare = DataPrepare(tag='valence', data=data, train_index=train_index, test_index=test_index, )
train_dataloader, test_dataloader = dataprepare.get_data()

(18089, 4, 400) (18089,) (4638, 4, 400) (4638,)
tensor(False) tensor(False) tensor(False) tensor(False)
